# Transport Plots of Model Data
*Akira Di Sandro, 7/1/20*
<br> In this notebook, I will be making some pseudocode for transport calculations plots of model data.

## 1. Import Packages as usual

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

## 2. Open Dataset

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [ ]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')

* (6/30) Oday is not available for 'uo' and 'vo'
* Also, I don't think I need 'wo' as a variable

* I think for the line above, I will need to find and download the CM4 dataset that I've been using but with a mask variable (look for it using variable) in order to do the transport calculations.
* (update from 6/30) for the mask, Raf said that I can just use NaN's as an indication that there is no data because of land so just using data from non-NaN suffices

In [ ]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

In [ ]:
grid = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
grid

* The above two lines will also be dependent on the change in line 4.

In [ ]:
dog1 = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omfx', variable_id='areacello')
dset_dict1 = dog1.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

In [ ]:
grid1 = dset_dict1['CMIP.NOAA-GFDL.GFDL-CM4.historical.r1i1p1f1.Ofx.areacello.gn']

## 3. Open Cluster 

In [ ]:
from dask.distributed import Client
from dask_gateway import Gateway


gateway = Gateway()  # connect to Gateway

cluster = gateway.new_cluster()  # create cluster
cluster.scale(10)  # scale cluster

client = Client(cluster)  # connect Client to Cluster

In [ ]:
client

## 4. Actual Calculations

### a. Calculations to get the U/V points

* (6/30) still need to edit these lat and lon to match the observational data

In [ ]:
lat1 = -4.49531
lon1 = -208.21236
lat2 = -5.989064
lon2 = -205.234668

In [ ]:
isec, jsec, xsec, ysec = sectionate.create_section(grid['lon'], grid['lat'], lon1, lat1, lon2, lat2)

In [ ]:
uvpoints = sectionate.transports_C.MOM6_UVpoints_from_section(isec, jsec)
udata = []
vdata = []

for point in uvpoints:
    pttype, i, j = point
    i = int(i)
    j = int(j)
    if pttype == 'U':
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values    
    udata = np.concatenate([udata, [i, j, lon, lat]], axis=1)  #this line needs to be tested out
    elif pttype == 'V':
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
    vdata = np.concatenate([vdata, [i, j, lon, lat]], axis=1)  #this line needs to be tested out
#    print(f'{point[0]}, {point[1]}, {point[2]}, {lon}, {lat}')

udata should be a 4-by-uvpoints that are u, matrix-like dataset that stores the i and j values and the corresponding longitude and latitude values for the grid point for which we care about the u point.
<br>udata = [[i values],
<br>         [j values],
<br>         [lon],
<br>         [lat]      ]
         
vdata should be a 4-by-uvpoints that are v, matrix-like dataset that stores the i and j values and the corresponding longitude and latitude values for the grid point for which we care about the v point.
<br>vdata = [[i values],
<br>         [j values],
<br>         [lon],
<br>         [lat]      ]

### b. Using U/V points to calculate transport
The following are the steps I cam eup with to do the calculation. <br>
* (transport for each cell) T = u * SA (of cell face) * mask
* SA = dy (vertical distance of cell) * depth of cell     (6/30) SA can be replaced with areacello
* Mask = fraction that tells us how much of the cell is in water (mostly 1 but for the edges, it will be some fraction<1)
* Do this calculation for all depths of (i,j) cell and summate


In [ ]:
# masked area code from Raf's template
#    this takes care of multiplying the SA by the mask, so all we have to do is use this area 
#    data for transport calculations
area_masked = grid1['areacello'].where(~np.isnan(ds['uo'].isel(time=0)))

### c. Using the above code to make a function that calculates the transport for each time stamp

In [ ]:
def calc_model_transport(ds, t_step, udata, vdata, mask):
    total = []
    for pt in udata:
        i,j,lon,lat = pt
        u = ds['uo'].isel(x=i,y=j,time=t_step).values
        SAm = mask.isel(x=i,y=j).values
        pt_total=0
        for row in range(len(SAm)):
            pt_total += u[row] * SAm[row]
        total = np.append(total, pt_total)
    utrans = np.sum(total)
    total = []
    for pt in vdata:
        i,j,lon,lat = pt
        v = ds['vo'].isel(x=i,y=j,time=t_step).values
        SAm = mask.isel(x=i,y=j).values
        pt_total=0
        for row in range(len(SAm)):
            pt_total += v[row] * SAm[row]
        total = np.append(total, pt_total)
    vtrans = np.sum(total)
    transport = utrans + vtrans
    return transport

## 5. Plot time series

I still don't know how I will deal with the fact that this model output only has monthly data while the observations are daily.

In [ ]:
# create array that stores time stamps I'm interested in (July 23, 2012 through March 4, 2014)
times = grid.time[slice('2012-07-23', '2014-03-04')]
#times = grid['time'].sel(slice('2012-07-23', '2014-03-04'))
#times = grid['time'].sel(time=slice('2012-07-23', '2014-03-04'))

# store transport for each time stamp in array above
transport = []
for i in range(len(times)):
    tp = calc_model_transport(grid, times[i], udata, vdata, mask)
    transport = np.append(transport, tp)

In [ ]:
# use similar method to obs data to make time series for model data
fig, ax = plt.subplots(figsize=(14,8))
ax.set_ylabel('Model Data Transport (Sv)')
ax.set_xlabel('Time')
ax.plot(times, transport)
ax.set_xlim(times[0], times[-1])
ax.grid(True)
plt.title('Monthly Time series of Model Data Transport', fontsize=20)
trans1_plot = plt.show()